<a href="https://colab.research.google.com/github/fangxiang00/Awesome-Chinese-NLP/blob/master/pytorch5_data_parallel.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

在多GPU上执行正向和反向传播是自然而然的事。然而，PyTorch默认将只是用一个GPU。你可以使用DataParallel让模型并行运行来轻易的在多个GPU上运行你的操作。

In [3]:
import torch
device=torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(device)

cpu


In [4]:
# 导入pytorch模块和定义参数
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader

# Parameters 和 DataLoaders
input_size = 5 #输入数据维度
output_size = 2 #输出数据维度

batch_size = 30
data_size = 100

In [5]:
#制作虚拟数据集
class MyDataset(Dataset):
  def __init__(self,input_size,data_size):
    self.len=data_size
    self.data=torch.randn(data_size,input_size)
  def __getitem__(self,index):
    return self.data[index]
  def __len__(self):
    return self.len

rand_loader=DataLoader(dataset=MyDataset(input_size,data_size),batch_size=batch_size,shuffle=True)

In [6]:
class Model(nn.Module):
  def __init__(self,input_size,output_size):
    super(Model,self).__init__()
    self.fc=nn.Linear(input_size,output_size)
  def forward(self,input):
    output=self.fc(input)
    print("\tIn Model: input size", input.size(),
          "output size", output.size())
    return output

这是本教程的核心部分。首先，我们需要创建一个模型实例和检测我们是否有多个GPU。如果我们有多个GPU，我们使用nn.DataParallel来包装我们的模型。然后通过model.to(device)把模型放到GPU上。

In [7]:
model = Model(input_size, output_size)
if torch.cuda.device_count() > 1: 
  print("Let's use", torch.cuda.device_count(), "GPUs!")
  # dim = 0 [30, xxx] -> [10, ...], [10, ...], [10, ...] on 3 GPUs
  model = nn.DataParallel(model)

model.to(device)

Model(
  (fc): Linear(in_features=5, out_features=2, bias=True)
)

In [8]:
for data in rand_loader: 
    input = data.to(device)
    output = model(input)
    print("Outside: input size", input.size(),
          "output_size", output.size())

	In Model: input size torch.Size([30, 5]) output size torch.Size([30, 2])
Outside: input size torch.Size([30, 5]) output_size torch.Size([30, 2])
	In Model: input size torch.Size([30, 5]) output size torch.Size([30, 2])
Outside: input size torch.Size([30, 5]) output_size torch.Size([30, 2])
	In Model: input size torch.Size([30, 5]) output size torch.Size([30, 2])
Outside: input size torch.Size([30, 5]) output_size torch.Size([30, 2])
	In Model: input size torch.Size([10, 5]) output size torch.Size([10, 2])
Outside: input size torch.Size([10, 5]) output_size torch.Size([10, 2])
